In [ ]:
### 해당 코드는 YOLOv5 형식의 폴리곤을 bounding box 형식으로 바꿔줍니다.
### TXT 폴더에 기존 label(annotation) 파일을 넣어주고 images 폴더에 원본 그림파일을 넣어줍니다.


import os
import shutil

# 폴더 경로 설정
txt_dir = "./TXT"
images_dir = "./images"

# 5개 이상의 값이 있는 라벨 파일을 변환하는 함수
def convert_to_bounding_box(file_path):
    converted_lines = []
    with open(file_path, "r") as f:
        for line in f:
            values = line.strip().split()
            if len(values) > 5:
                # 클래스 ID는 그대로 사용하고, 나머지 좌표값들을 float형으로 변환하여 좌표 리스트로 변환
                class_id = values[0]
                coordinates = list(map(float, values[1:]))

                # x 좌표와 y 좌표를 각각 추출하여 min/max 값을 찾음
                x_coords = coordinates[::2]  # x 좌표만 추출
                y_coords = coordinates[1::2]  # y 좌표만 추출
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)

                # 중심 좌표(x_center, y_center)와 너비(width), 높이(height) 계산
                x_center = (x_min + x_max) / 2
                y_center = (y_min + y_max) / 2
                width = x_max - x_min
                height = y_max - y_min

                # 변환된 라벨 형식으로 저장
                converted_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                converted_lines.append(converted_line)
            else:
                # 이미 5개의 값인 경우는 그대로 추가
                converted_lines.append(line)
    return converted_lines

# 변환 전 TXT 폴더의 파일 개수 확인
txt_files_before = sorted([f for f in os.listdir(txt_dir) if f.endswith(".txt")])
txt_count_before = len(txt_files_before)

# 변환된 데이터를 원본 파일에 덮어쓰기
for filename in txt_files_before:
    file_path = os.path.join(txt_dir, filename)
    # 각 파일의 변환 결과 가져오기
    converted_data = convert_to_bounding_box(file_path)

    # 변환된 데이터를 TXT 폴더 내 원본 파일에 덮어쓰기
    with open(file_path, "w") as f:
        f.writelines(converted_data)

# 변환 후 TXT 폴더와 images 폴더의 파일 개수 및 파일명 비교
txt_files_after = sorted([f for f in os.listdir(txt_dir) if f.endswith(".txt")])
image_files = sorted([f for f in os.listdir(images_dir) if os.path.splitext(f)[1] in ['.jpg', '.jpeg', '.png', '.bmp', '.gif']])

txt_count_after = len(txt_files_after)
image_count = len(image_files)

# 변환 전과 변환 후의 TXT 파일 개수 출력
print(f"변환 전 TXT 폴더 내 파일 개수: {txt_count_before}")
print(f"변환 후 TXT 폴더 내 파일 개수: {txt_count_after}")
print(f"images 폴더 내 파일 개수: {image_count}")

# 파일 개수와 이름 확인
if txt_count_after == image_count and txt_files_after == [os.path.splitext(f)[0] + ".txt" for f in image_files]:
    print("TXT 폴더와 images 폴더의 파일 개수 및 파일명이 동일합니다.")
    print("변환된 TXT 파일을 기존 TXT 파일에 덮어쓰기 했습니다.")
else:
    print("경고: TXT 폴더와 images 폴더의 파일 개수 또는 파일명이 다릅니다.")


변환 전 TXT 폴더 내 파일 개수: 246
변환 후 TXT 폴더 내 파일 개수: 246
images 폴더 내 파일 개수: 246
TXT 폴더와 images 폴더의 파일 개수 및 파일명이 동일합니다.
변환된 TXT 파일을 기존 TXT 파일에 덮어쓰기 했습니다.
